In [1]:
import pyodbc
import pymssql
import pandas as pd
import urllib.parse
from sqlalchemy import create_engine
pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [13]:
host = '10.0.4.20'
nombre_bd = 'Campus'
nombre_usuario = 'dbhz'
password = 'Horarios.5822@'
password = urllib.parse.quote_plus(password)

conexion = create_engine("mssql+pymssql://"+nombre_usuario+":"+password+"@"+host+"/"+nombre_bd,deprecate_large_types=True)

periodo_consulta  = '2022'
semestre_consulta = 'SEM2'


In [14]:
consulta_result_materias = """
--declare @periodo_consulta varchar(4),
--	@semestre_consulta varchar(4)

Select
	p.SUFFIX as TipoDoc,
	p.NICKNAME as Identificacion,
	p.people_code_id,
	p.FIRST_NAME + ' ' + p.MIDDLE_NAME + ' ' + p.LAST_NAME as Nombres,
	org.org_name_1 as sede,
	cu.LONG_DESC as ProgramaEstudiante,
	fac.long_desc as Facultad,
	cu.STATUS as EstadoAlumnoPrograma,
	Ac.CLASS_LEVEL Semestre,
	TaDe.ACADEMIC_YEAR as año,
	TaDe.ACADEMIC_TERM as Periodo,
	TaDe.ACADEMIC_SESSION as Sesion,
	-- Tade.REVISION_DATE FechaMatricula,
	Secc.CodigoMateria,
	SeCc.NombreMateria,
	Secc.Seccion,
	tg.GRADE_ACTIVITY,
	tg.GRADE_POINTS,
	tade.FINAL_GRADE,
	Secc.ProgramaMateria,
	--tade.FINAL_GRADE,
	(
		select
			ev.long_desc
		from
			CODE_EVENTSUBTYPE ev
		where
			ev.code_value_key = secc.EVENT_SUB_TYPE
	) as TipoMateri,
(
		select
			top 1 dir.address_line_1 + ' ' + dir.city
		from
			Addressschedule dir
		where
			P.People_code_Id = dir.People_Org_Code_Id
			and Address_type = 'FAMI'
	) DIRECCION,
	(
		select
			top 1 ciu.long_desc
		from
			Addressschedule dir,
			code_county ciu
		where
			ciu.code_value_key = dir.county
			and P.People_code_Id = dir.People_Org_Code_Id
			and Address_type = 'FAMI'
	) CIUDAD,
	(
		select
			top 1 dep.long_desc
		from
			Addressschedule dir,
			code_state dep
		where
			dep.code_value_key = dir.state
			and P.People_code_Id = dir.People_Org_Code_Id
			and Address_type = 'FAMI'
	) DEPARTAMENTO,
(
		select
			top 1 Ppf.PhoneNumber
		from
			PersonPhone as Ppf
		where
			Ppf.PersonId = P.PersonId
			and ppf.PhoneType = 'FAMILIAR'
	) TelFijo,
(
		select
			top 1 Ppc.PhoneNumber
		from
			PersonPhone as Ppc
		where
			Ppc.PersonId = P.PersonId
			and ppc.PhoneType = 'CELULAR'
	) TelMovil,
(
		select
			top 1 dir.EMAIL_ADDRESS
		from
			Addressschedule dir
		where
			P.People_code_Id = dir.People_Org_Code_Id
			and Address_type = 'FAMI'
	) EMAIL,
	dm.GENDER as Genero,
(
		select
			top 1 DISCIPLINE
		from
			DEGREQEVENT as de
		where
			tade.EVENT_ID = de.EVENT_ID
			and ac.CURRICULUM = de.CURRICULUM
	) seme,
	p1.PEOPLE_CODE_ID as Cog_Docente,
	p1.nickname Identificacion4,
	p1.LAST_NAME + ' ' + p1.FIRST_NAME + ' ' + p1.MIDDLE_NAME as Nom_Docente,
	(
		select
			TOP 1 d.Tipo_de_vinculacion
		from
			[DB-HZ].dbo.USUARIOS u
			inner join [DB-HZ].dbo.DOCENTES d on d.Id_USUARIO = u.Id
		where
			CONVERT(VARCHAR, u.CEDULA) = p1.nickname
	) Vinculación_Docente,
	(
		select
			TOP 1 NOMBRE_DEPARTAMENTO
		from
			[DB-HZ].dbo.USUARIOS u
			inner join [DB-HZ].dbo.DOCENTES d on d.Id_USUARIO = u.Id
			inner join [DB-HZ].dbo.DEPARTAMENTOS dp on d.Departamento = dp.Id
		where
			CONVERT(VARCHAR, u.CEDULA) = p1.nickname
	) DEPARTAMENTO,
	case
		when ac.CLASS_LEVEL in ('SE01') then '1'
		when ac.CLASS_LEVEL in ('SE02') then '2'
		when ac.CLASS_LEVEL in ('SE03') then '3'
		when ac.CLASS_LEVEL in ('SE04') then '4'
		when ac.CLASS_LEVEL in ('SE05') then '5'
		when ac.CLASS_LEVEL in ('SE06') then '6'
		when ac.CLASS_LEVEL in ('SE07') then '7'
		when ac.CLASS_LEVEL in ('SE08') then '8'
		when ac.CLASS_LEVEL in ('SE09') then '9'
		when ac.CLASS_LEVEL in ('SE10') then '10'
		when ac.CLASS_LEVEL in ('NULL') then '0'
		when ac.CLASS_LEVEL in ('') then '0'
	end as SemNumero ------------------
,
	isnull(
		convert(numeric(9, 1), round(tg.grade_points, 2, 1)),
		0
	) Nota1 ----------
,
	case
		when tg.GRADE_POINTS >= '3' then '1'
		when tg.GRADE_POINTS < '3' then '0'
	end as Gano ----------------
	--tener en cuenta los null en el excel para las perdidas o ganadas (1 o 0)
	----------------
,
	case
		when tg.GRADE_POINTS < '3' then '1'
		when tg.GRADE_POINTS >= '3' then '0'
	end as Perdio --- convertir los null manual---
,
	case
		when tg.GRADE_POINTS = '0' then '0. Cero'
		when (
			tg.GRADE_POINTS > '0'
			and tg.GRADE_POINTS < '2'
		) then '1. Muy Baja'
		when (
			tg.GRADE_POINTS >= '2'
			and tg.GRADE_POINTS < '3'
		) then '2. Baja'
		when (
			tg.GRADE_POINTS >= '3'
			and tg.GRADE_POINTS < '4'
		) then '3. Media'
		when (
			tg.GRADE_POINTS >= '4'
			and tg.GRADE_POINTS < '4.5'
		) then '4. Alta'
		when (
			tg.GRADE_POINTS >= '4.5'
			and tg.GRADE_POINTS <= '5'
		) then '5. Muy Alta'
	end as Rango -------------------
,
(
		select
			top 1 case
				when de.DISCIPLINE = '01SEMESTRE' then '1'
				when de.DISCIPLINE = '02SEMESTRE' then '2'
				when de.DISCIPLINE = '03SEMESTRE' then '3'
				when de.DISCIPLINE = '04SEMESTRE' then '4'
				when de.DISCIPLINE = '05SEMESTRE' then '5'
				when de.DISCIPLINE = '06SEMESTRE' then '6'
				when de.DISCIPLINE = '07SEMESTRE' then '7'
				when de.DISCIPLINE = '08SEMESTRE' then '8'
				when de.DISCIPLINE = '09SEMESTRE' then '9'
				when de.DISCIPLINE = '10SEMESTRE' then '10'
				when de.DISCIPLINE = 'NULL' then '0'
				when de.DISCIPLINE = ' ' then '0'
			end
		from
			DEGREQEVENT as de
		where
			tade.EVENT_ID = de.EVENT_ID
			and ac.CURRICULUM = de.CURRICULUM
	) SemMateriaNum,
(3.2 - GRADE_POINTS * 1 / 3) * 3 / 2 as ProxNotaMin,
	secc.Creditos, ac.curriculum
From
	PEOPLE p
	inner join ACADEMIC ac on p.PEOPLE_CODE_ID = ac.PEOPLE_CODE_ID
	inner join CODE_CURRICULUM cu on ac.CURRICULUM = cu.CODE_VALUE_KEY
	inner join TRANSCRIPTDETAIL TaDe on p.PEOPLE_CODE_ID = TaDe.PEOPLE_CODE_ID
	and ac.ACADEMIC_YEAR = Tade.ACADEMIC_YEAR
	and ac.ACADEMIC_TERM = TaDe.ACADEMIC_TERM
	and ac.ACADEMIC_SESSION = TaDe.ACADEMIC_SESSION
	and ac.transcript_seq = tade.transcript_seq
	inner join (
		Select
			sec.SECTION as Seccion,
			Eve.EVENT_ID as CodigoMateria,
			sec.EVENT_SUB_TYPE,
			Eve.EVENT_LONG_NAME as NombreMateria,
			Cur.LONG_DESC as ProgramaMateria,
			sec.EVENT_STATUS as EstadoSeccion,
			Sec.ACADEMIC_YEAR as Año,
			Sec.ACADEMIC_TERM as Periodo,
			Sec.ACADEMIC_SESSION as Sesion,
			sec.CREDITS as Creditos
		From
			SECTIONS Sec
			left outer join CODE_CURRICULUM Cur on Sec.CURRICULUM = Cur.CODE_VALUE_KEY
			inner join EVENT Eve on Sec.EVENT_ID = Eve.EVENT_ID
	) SeCc on TaDe.SECTION = Secc.Seccion
	and TaDe.EVENT_ID = secc.CodigoMateria
	and TaDe.ACADEMIC_YEAR = Secc.Año
	and TaDe.ACADEMIC_TERM = Secc.Periodo
	and TaDe.ACADEMIC_SESSION = secc.Sesion
	and tade.EVENT_SUB_TYPE = secc.EVENT_SUB_TYPE
	left outer join TRANSCRIPTGRADING tg on tg.PEOPLE_CODE_ID = p.PEOPLE_CODE_ID
	and tg.PEOPLE_CODE_ID = tade.PEOPLE_CODE_ID
	and tg.ACADEMIC_YEAR = tade.ACADEMIC_YEAR
	and tg.ACADEMIC_TERM = tade.ACADEMIC_TERM
	and tg.ACADEMIC_SESSION = tade.ACADEMIC_SESSION
	and tg.EVENT_ID = tade.EVENT_ID
	and tg.EVENT_SUB_TYPE = tade.EVENT_SUB_TYPE
	and tg.SECTION = tade.SECTION
	left outer join code_college as fac on ac.COLLEGE = fac.CODE_VALUE_KEY
	left outer join demographics as dm on ac.people_code_id = dm.people_code_id
	and ac.ACADEMIC_YEAR = dm.ACADEMIC_YEAR
	and ac.ACADEMIC_TERM = dm.ACADEMIC_TERM
	and ac.ACADEMIC_SESSION = dm.ACADEMIC_SESSION
	left outer join organization as org on org.org_code_id = ac.org_code_id
	left outer join SECTIONPER sp on TaDe.ACADEMIC_YEAR = sp.ACADEMIC_YEAR
	and TaDe.ACADEMIC_TERM = sp.ACADEMIC_TERM
	and TaDe.ACADEMIC_SESSION = sp.ACADEMIC_SESSION
	and TaDe.EVENT_ID = sp.EVENT_ID
	and TaDe.SECTION = sp.SECTION
	left outer join PEOPLE p1 on p1.PEOPLE_CODE_ID = sp.PERSON_CODE_ID
where
	ac.ACADEMIC_YEAR = %s
	and ac.ACADEMIC_TERM = %s
	and ac.ACADEMIC_SESSION = 'Preg01'
	and ac.academic_flag = 'Y'
	and tade.ADD_DROP_WAIT = 'A'
	and secc.EstadoSeccion in ('A','P')
order by
	fac.long_desc,
	cu.LONG_DESC,
	p.PEOPLE_CODE_ID

"""

consulta_cancelaciones = """

SELECT DISTINCT P.PEOPLE_CODE_ID ID_SISTEMA, P.SUFFIX TIPO_DOCUMENTO, P.NICKNAME NUM_DOCUMENTO, (p.LAST_NAME+' '+p.FIRST_NAME+' '+p.MIDDLE_NAME) ESTUDIANTE,(A.ACADEMIC_YEAR+' - '+A.ACADEMIC_TERM+' - '+A.ACADEMIC_SESSION) PERIODO_ACADEMICO, CCF.LONG_DESC FACULTAD, CCD.LONG_DESC DEPARTAMENTO,
       CC.LONG_DESC PROGRAMA, A.CLASS_LEVEL SEMESTRE_ESTUDIANTE,O.ORG_NAME_1 LUGAR_DE_DESARROLLO, TM.EVENT_ID ID_ASIGNATURA_CANCELADA, E.EVENT_LONG_NAME NOMBRE_ASIGNATURA, TM.SECTION GRUPO, TM.DROP_REASON ESTADO, DE.EVENT_CLASS TIPO_ASIGNATURA,
       DE.DISCIPLINE, A.curriculum
FROM ACADEMIC A
INNER JOIN 
PEOPLE P ON P.PEOPLE_CODE_ID = A.PEOPLE_CODE_ID 
INNER JOIN 
CODE_COLLEGE CCF ON CCF.CODE_VALUE_KEY = A.COLLEGE
INNER JOIN 
CODE_DEPARTMENT CCD ON CCD.CODE_VALUE_KEY = A.DEPARTMENT
INNER JOIN 
CODE_CURRICULUM CC ON CC.CODE_VALUE_KEY = A.CURRICULUM
INNER JOIN ORGANIZATION O ON O.ORG_CODE_ID = A.ORG_CODE_ID
INNER JOIN 
TRANSCRIPTMARKETING TM ON TM.PEOPLE_CODE_ID = A.PEOPLE_CODE_ID 
AND A.ACADEMIC_YEAR = TM.ACADEMIC_YEAR
AND A.ACADEMIC_TERM = TM.ACADEMIC_TERM
AND A.ACADEMIC_SESSION = TM.ACADEMIC_SESSION
AND DROP_REASON  = 'CancMateri' -- in ('CancMateri',''CancSemest)
INNER JOIN 
EVENT E ON E.EVENT_ID = TM.EVENT_ID
INNER JOIN 
DEGREQEVENT DE ON DE.EVENT_ID = E.EVENT_ID
AND DE.MATRIC_YEAR = A.MATRIC_YEAR
AND DE.MATRIC_TERM = A.MATRIC_TERM
AND DE.CURRICULUM = A.CURRICULUM


WHERE A.ACADEMIC_YEAR = %s
AND A.ACADEMIC_TERM = %s
AND A.ACADEMIC_SESSION = 'PREG01'
AND A.ACADEMIC_FLAG = 'Y'
AND A.ENROLL_SEPARATION IN ('5','12')

/* Sacar Laboratorios */
AND A.PEOPLE_CODE_ID NOT IN (SELECT PEOPLE_CODE_ID 
                             FROM TRANSCRIPTDETAIL
                             WHERE ACADEMIC_YEAR = A.ACADEMIC_YEAR
								AND ACADEMIC_TERM = A.ACADEMIC_TERM
								AND ACADEMIC_SESSION = A.ACADEMIC_SESSION
								AND EVENT_ID = TM.EVENT_ID
								AND SECTION = TM.SECTION
								AND (SECTION LIKE 'L%' AND CREDIT = 0))
ORDER BY PROGRAMA,ID_SISTEMA, TM.EVENT_ID

"""


In [15]:
resul_materias = pd.read_sql(consulta_result_materias,conexion, params=[periodo_consulta,semestre_consulta])

cancelaciones = pd.read_sql(consulta_cancelaciones,conexion, params = [periodo_consulta,semestre_consulta])

In [77]:
resul_materias['concatenacion'] = resul_materias['people_code_id'].str.cat(resul_materias['ProgramaEstudiante'], sep='-')

In [83]:
cancelaciones.head()

,ID_SISTEMA,TIPO_DOCUMENTO,NUM_DOCUMENTO,ESTUDIANTE,PERIODO_ACADEMICO,FACULTAD,DEPARTAMENTO,PROGRAMA,SEMESTRE_ESTUDIANTE,LUGAR_DE_DESARROLLO,ID_ASIGNATURA_CANCELADA,NOMBRE_ASIGNATURA,GRUPO,ESTADO,TIPO_ASIGNATURA,DISCIPLINE,curriculum
0,P000101689,CC,1003721322,VARGAS CASARRUBIA MIGUEL ALFONSO,2022 - SEM1 - PREG01,Fac De Medicina Veterinaria y Zootecnia,Departamento de Ciencias Acuícolas,Acuicultura,SE04,Campus Montería,103249,DISEÑO EXPERIMENTAL,G1,CancMateri,Obligatori,04SEMESTRE,908
1,P000102075,CC,1233342055,LEON MONTES MANOLO JOSE,2022 - SEM1 - PREG01,Fac De Medicina Veterinaria y Zootecnia,Departamento de Ciencias Acuícolas,Acuicultura,SE07,Campus Montería,103126,ADMINISTRACION AGROPECUARIA,G1,CancMateri,Obligatori,08SEMESTRE,908
2,P000106337,CC,1003097745,CUADRADO WILCHES ELIZABETH,2022 - SEM1 - PREG01,Fac De Medicina Veterinaria y Zootecnia,Departamento de Ciencias Acuícolas,Acuicultura,SE03,Campus Montería,103239,SUELOS,G1,CancMateri,Obligatori,03SEMESTRE,908
3,P000114766,CC,1003187394,PERNETH LOPEZ EDUARDO ALFONSO,2022 - SEM1 - PREG01,Fac De Medicina Veterinaria y Zootecnia,Departamento de Ciencias Acuícolas,Acuicultura,SE03,Campus Montería,103249,DISEÑO EXPERIMENTAL,G1,CancMateri,Obligatori,04SEMESTRE,908
4,P000119294,CC,1067966126,DELGADO DORIA VICTOR ALBERTO,2022 - SEM1 - PREG01,Fac De Medicina Veterinaria y Zootecnia,Departamento de Ciencias Acuícolas,Acuicultura,SE05,Campus Montería,103249,DISEÑO EXPERIMENTAL,G1,CancMateri,Obligatori,04SEMESTRE,908


In [84]:
def calculo_var(estudiante):
    base = resul_materias[resul_materias['concatenacion']==estudiante]
    
    num_materias =  base['CodigoMateria'].nunique()
    
    num_materias_can = cancelaciones[(cancelaciones['ID_SISTEMA']==estudiante.split('-')[0]) & (cancelaciones['PROGRAMA']==estudiante.split('-')[1])]['ID_ASIGNATURA_CANCELADA'].nunique()
    
    num_materias_gan = base.groupby('CodigoMateria')['GRADE_POINTS'].mean().apply(lambda x: 1 if x>=3 else 0).sum()
    

    estruc = [periodo_consulta,semestre_consulta,base['Identificacion'].unique()[0],estudiante,base['ProgramaEstudiante'].unique()[0],base['CIUDAD'].unique()[0],num_materias,num_materias_can,num_materias_gan]

    return estruc
   

In [85]:
lista_estud = resul_materias.copy()
lista_estud.drop_duplicates(['concatenacion'], inplace=True)

In [86]:
resultado = lista_estud['concatenacion'].apply(calculo_var)

In [87]:
# data = pd.DataFrame('resultado')
resultado.to_excel('data_snies_cor.xlsx')

In [ ]:
# data = pd.DataFrame(lista)
# data.columns = ['AÑO','SEMESTRE','NUM_DOCUMENTO','ID_ESTUDIANTE','PROGRAMA','CIUDAD','MATERIAS_MATRI','MATERIAS_CANCELADAS','MATERIAS_GANADAS']